# Experiments - Real, Friendship

In this notebook, we run experiments on a real network using the friendship-based method. In this method, users delegate in priority to people who have a lot of common friends (i.e. neighbours) with them.

Our real networks includes *Facebook*.

In [4]:
import rankedDelegation as rd
import numpy as np
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
from IPython.utils import io
np.random.seed(42)

## Load the data

The following cell show how to load a dataset.

In [5]:
path = ()
list_datasets = [{"path":"",
                  "name":"Facebook-reduced"}]
                 


def load_data(index):
    full_path = path + list_datasets[index]["path"]
    list_edges = np.genfromtxt(full_path, delimiter=" ",dtype=int)[:,:2]
    nb_nodes= int(np.max(list_edges))+1
    list_nodes = np.arange(nb_nodes)
    count_out = np.zeros(nb_nodes)
    count_in = np.zeros(nb_nodes)

    for (a, b) in tqdm(list_edges):
        if a != b:
            count_out[a] += 1
            count_in[b] += 1
    
    return {"edges":list_edges,"out": count_out, "in": count_in, "nb_nodes": nb_nodes}

## Saving the results

In [6]:
def save_election(e, results, title=""):
    L = []
    C = []
    timestamp = int(time.time())
    for v in tqdm(e.list_voters):
        for i, d in enumerate(v.delegatees):
            L.append((v.id, d.id, i+1))
        if v.vote is not None:
            C.append((v.id, 1))
        else:
            C.append((v.id, 0))
        
    np.savetxt("Save/edges_%s_%i.csv"%(title,timestamp), L)
    np.savetxt("Save/nodes_%s_%i.csv"%(title,timestamp), C)
    results.to_csv("Save/results_%s_%i.csv"%(title, timestamp))

## Running the experiments

We will run the experiments on wide ruleset, testing all the rules described on the paper (BFD, DFD, MinSum, Diffusion, LexiMax and BordaArborescence). We will look at a wide set of metrics:

* Maximum rank in a delegation path
* Maximum length of a delegation path
* Average length of the delegation paths
* Maximum Weight/Representation of a casting voter
* Power entropy
* Maximum sum of rank of a delegation path
* Sum of all rank in the arborescence (for confluent rules)
* Average rank in the arborescence (for confluent rules)
* Unpopularity of the arborescence (for confluent rules)

In [7]:
rules = [rd.rules.breadthFirst, rd.rules.minSum, rd.rules.depthFirst, 
             rd.rules.diffusion, rd.rules.lexiMax, rd.rules.bordaArb]
rules_str = ["BFD", "MinSum", "DFD","Diffusion", "LexiMax","BordaArb"]
metrics_str = ["MaxRank", "MaxLength", "AvgLength","MaxWeight", "MaxRep",
                "PowerEntropy", "MaxSum", "SumRank", "AvgRank", "Unpopularity"]
n_rules = len(rules)

In [8]:
def run_election(index):
    data= load_data(index)
    tab_results = np.zeros((10, n_rules))
    e = rd.election_friendship_real(data)

    for i, rule in tqdm(enumerate(rules)):
        e.attribute_gurus(rule)
        tab_results[0, i] += e.max_rank
        tab_results[1, i] += e.max_length
        tab_results[2, i] += e.mean_length
        tab_results[3, i] += e.max_power
        tab_results[4, i] += e.max_representation
        tab_results[5, i] += e.power_entropy
        tab_results[6, i] += e.max_sum
        tab_results[7, i] += e.sum_rank
        tab_results[8, i] += e.avg_rank
        tab_results[9, i] += e.unpopularity

    df = pd.DataFrame(tab_results.T, index=rules_str, columns=metrics_str)
    save_election(e, df, list_datasets[index]["name"])

#run_election(0)